In [25]:
import sacrebleu

def read_tgt_data(file):
    tgts=[]
    with open(file,'r') as fin:
        lines=fin.readlines()
        for line in lines:
            src,tgt=line.strip().split('￨')
            tgt_=tgt.split(' ')[1:]
            tgt=''
            for word in tgt_:
                tgt+=word
                tgt+=' '
            tgts.append(tgt[:-1])
    return tgts
def cal_bleu(ref_file,output_file,num=-1):
    refs=read_tgt_data(file=ref_file)
    outputs=read_tgt_data(file=output_file)
    if num!=-1:
        refs=refs[0:num]
        outputs=outputs[0:num]
    bleu = sacrebleu.corpus_bleu(outputs, [refs])
#     bleu = sacrebleu.corpus_bleu(refs , [outputs])
    
    return bleu.score


def eval_model(output_folder,output_file,decode_num):
    def get_ref_file(mode):
        """
        mode: [test,test_full,val,val_full]
        """
        ref_file='../data/src_data_full_feat_tf_resplited_review/{}_ref.txt'.format(mode)
        return ref_file
    def get_output_file(output_folder,output_file,mode):
        """
        mode: [test,test_full,val,val_full]
        """
        decode_file='{}_ref.txt'.format(mode)
        output_file='{}/{}_{}'.format(output_folder,decode_file,output_file)
        return output_file
    
    bleu={}
    print('='*30)
    print(output_file)
    for mode in ['test','val','train']:
        ref_mode=mode
        output_mode=mode
        ref_file=get_ref_file(ref_mode)
        output_file_=get_output_file(output_folder,output_file,output_mode)
        try:
            if mode=='train':
                bleu_score=cal_bleu(ref_file,output_file_,decode_num)
            else:
                bleu_score=cal_bleu(ref_file,output_file_)
                
            bleu[output_mode]=bleu_score
            print(output_mode,bleu_score)
        except:
            pass

def gen_decode_bash(model_folder,output_folder,nums,file,CUDA=0,decode_num=-1):
    with open(file,'w') as fout:
        for num in nums:

            for decode_file in ['test_ref.txt','val_ref.txt','train_ref.txt']:
                if decode_file=='train_ref.txt':
                    decode_num_=decode_num
                else:
                    decode_num_=-1
            
                num=str(num)
                bash="""CUDA_VISIBLE_DEVICES={CUDA} python gpt_decoder.py --model_folder '{model_folder}' \\
                --model_name GP2-pretrain-step-{num}.pkl \\
                --data_folder ../data/src_data_full_feat_tf_resplited_review \\
                --decode_file {decode_file} --decode_num {decode_num} \\
                --output_folder {output_folder} \\
                --output_file step-{num}.txt
                """.format(CUDA=CUDA,
                          model_folder=model_folder,
                          num=num,
                          decode_file=decode_file,
                           decode_num=decode_num_,
                           output_folder=output_folder,
                          )

                fout.write(bash+'\n')
    return

In [2]:
def get_bleu(nums,best_nums,folder,save_prefix):
    for num in nums:
        eval_model(output_folder='outputs/{}'.format(folder),output_file='{}{}.txt'.format(save_prefix,int(num)))
    for best_num in best_nums:
        eval_model(output_folder='outputs/{}'.format(folder),output_file='{}best_{}.txt'.format(save_prefix,int(best_num)))

In [17]:
cal_bleu(ref_file='../data/src_data_full_feat_tf_resplited_review/test_ref.txt',output_file='../outputs/medium_10epochs_trial_2/test_ref.txt_step-12500.txt')

0.11703895582180705

In [19]:
cal_bleu(ref_file='../data/src_data_full_feat_tf_resplited_review/test_ref.txt',output_file='../outputs/medium_10epochs_trial_2/test_ref.txt_step-37500.txt')

0.17393276928908522

In [21]:
cal_bleu(ref_file='../data/src_data_full_feat_tf_resplited_review/test_ref.txt',output_file='../outputs/medium/test_ref.txt_medium_ft.txt')

0.03615903691369294

In [23]:
cal_bleu(ref_file='../data/src_data_full_feat_tf_resplited_review/test_ref.txt',output_file='../outputs/medium_10epochs/test_ref.txt_step-85000.txt')

0.15148506459777716

In [18]:
cal_bleu(ref_file='../data/src_data_full_feat_tf_resplited_review/val_ref.txt',output_file='../outputs/medium_10epochs_trial_2/val_ref.txt_step-12500.txt')

0.10211872327674586

In [18]:
cal_bleu(ref_file='../data/src_data_full_feat_tf_resplited_review/val_full_ref.txt',output_file='../outputs/medium_10epochs_trial_2/val_full_ref.txt_step-12500.txt')

0.10211872327674586

In [24]:
cal_bleu(ref_file='../data/src_data_full_feat_tf_resplited_review/test_full_ref.txt',output_file='../outputs/medium_10epochs_trial_2/test_full_ref.txt_step-12500.txt')

0.11664429560219294

In [ ]:
CUDA_VISIBLE_DEVICES=1 python gpt_decoder.py --model_folder '../models/medium_10epochs_trial_2/GPT2.1e-05.20.1gpu.2020-04-15200256' \
--model_name GP2-pretrain-step-37500.pkl \
--data_folder ../data/src_data_full_feat_tf_resplited_review \
--decode_file test_ref.txt \
--output_folder ../outputs/medium_10epochs_trial_2 \
--output_file step-37500.txt


CUDA_VISIBLE_DEVICES=0 python gpt_decoder.py --model_folder '../models/medium' \
--model_name medium_ft.pkl \
--data_folder ../data/src_data_full_feat_tf_resplited_review \
--decode_file test_ref.txt \
--output_folder ../outputs/medium \
--output_file medium_ft.txt


CUDA_VISIBLE_DEVICES=0 python gpt_decoder.py --model_folder '../models/medium_10epochs/GPT2.1e-05.20.1gpu.2020-04-15053755' \
--model_name GP2-pretrain-step-85000.pkl \
--data_folder ../data/src_data_full_feat_tf_resplited_review \
--decode_file test_ref.txt \
--output_folder ../outputs/medium_10epochs \
--output_file step-85000.txt

CUDA_VISIBLE_DEVICES=0 python gpt_decoder.py --model_folder '../models/medium_10epochs_trial_3/GPT2.1e-05.20.1gpu.2020-04-16050142' \
--model_name GP2-pretrain-step-65000.pkl \
--data_folder ../data/src_data_full_feat_tf_resplited_review \
--decode_file test_ref.txt \
--output_folder ../outputs/medium_10epochs_trial_3 \
--output_file step-65000.txt




In [27]:
model_folder='../models/medium_10epochs_trial_3/GPT2.1e-05.20.1gpu.2020-04-16050142'
steps=[70000,45000,22500]
steps=[210000,187500]
# steps=[232500]
steps=[250000,232500]
CUDA=1
decode_num=10000
output_folder='../outputs/medium_10epochs_trial_3'
file='eval_medium_2.sh'

gen_decode_bash(model_folder,output_folder,steps,file,CUDA,decode_num)



In [33]:
steps=[250000,232500]
for step in steps:
    eval_model(output_folder='../outputs/medium_10epochs_trial_3',
               output_file='step-{}.txt'.format(step),
               decode_num=decode_num)

step-250000.txt
test 1.1667799812568302
val 1.1294278609381674
train 5.121912927493648
step-232500.txt
test 1.0877187687709555
val 1.0941272553621735
train 3.842732962699449


In [34]:
steps=[i*10000 for i in range(2,18,2)]

In [35]:
steps

[20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000]

In [46]:
model_folder='../../DialoGPT/models/medium_10epochs_trial_4/GPT2.1e-05.20.1gpu.2020-04-18015154'
# steps=[20000, 40000, 60000, 80000]
# steps=[100000, 120000, 140000, 160000]
steps=[180000,200000]
# steps=[220000,240000]
steps=[250000]

CUDA=0
decode_num=10000
output_folder='../outputs/medium_10epochs_trial_4'
file='eval_medium_1.sh'

gen_decode_bash(model_folder,output_folder,steps,file,CUDA,decode_num)



In [41]:
decode_num=10000
output_folder='../outputs/medium_10epochs_trial_4'
file='eval_medium_2.sh'

steps=[20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000]
for step in steps:
    eval_model(output_folder=output_folder,
               output_file='step-{}.txt'.format(step),
               decode_num=decode_num)

step-20000.txt
test 0.8048180739337994
val 0.8803899843338444
train 7.802846712614786
step-40000.txt
test 1.0011792509399924
val 1.0103775179238799
train 4.870033467099847
step-60000.txt
test 1.1494182080539148
val 1.4309372612863296
train 3.6049569509655695
step-80000.txt
test 1.5136761670170067
val 1.6869182456059462
train 4.737564179843113
step-100000.txt
test 1.5158436582862727
val 1.5907977456464772
train 5.507689345332864
step-120000.txt
test 1.692008112806717
val 1.8740326764680773
train 5.707224655890369
step-140000.txt
test 1.8452942124974379
val 1.995035680889552
train 5.92835729838237
step-160000.txt
test 1.9598692829303723
val 2.2250285210417746
train 7.718717600857212


In [47]:
decode_num=10000
output_folder='../outputs/medium_10epochs_trial_4'
file='eval_medium_2.sh'

steps=[250000,180000,200000,220000,240000]
for step in steps:
    eval_model(output_folder=output_folder,
               output_file='step-{}.txt'.format(step),
               decode_num=decode_num)

step-250000.txt
test 2.113877351821141
val 2.5586963235763065
train 6.475645349541125
step-180000.txt
test 1.9492704817342041
val 2.259773587510747
train 6.150700614908794
step-200000.txt
test 2.080680181507942
val 2.467184719632024
train 7.7089076904815
step-220000.txt
test 2.541667008819786
val 2.5455910411181217
train 7.733149283996843
step-240000.txt
test 2.2603283716266436
val 2.60454242501654
train 6.714508499505851
